# Section 5: BIM e Innovación

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pandas as pd
from itertools import zip_longest as zip
from itertools import count
import numpy as np
import codecs

## A. Writing Json file with format 
def write_json_UTF8(jsonDir, name_file):
    with open(name_file, 'w') as outfile:
        json.dump(jsonDir, outfile, ensure_ascii=False)
    with codecs.open(name_file, 'r', encoding = "iso-8859-1") as file:
          lines = file.read()
    with codecs.open(name_file, 'w', encoding = 'utf8') as file:
          file.write(lines)
    print("CREATED: ", name_file)
    
## B. Fixing numbers             
def repair_num(value):
    try:
        aux = float(value)
    except:
        "Existe algun signo"
        aux = ""
        for letra in str(value):

            if letra.isdigit():
                aux = aux+letra
            elif letra == ".":
                aux = aux+letra
        if aux == "":
            pass

    return float(aux)

## C. Formating Json
def DataToJson(df, varible_v, variable_meaning, reg_name, toolTip_v, toolTip_mening, norm=False):
    globalDir = {}
    anioInit = -999
    listData = []
    dirData = {}
    initDir = True
    df_norm = None
    if norm:
        df_copy = df.copy()
        df_norm = (df_copy[varible_v]-df_copy[varible_v].min()) / \
            (df_copy[varible_v].max()-df_copy[varible_v].min())

    for i, anio, mes, var_value in zip(count(), df["anio"], df["mesSTR"], df[varible_v]):
        try:
            anio = int(anio)
        except:
            print("set:", anio)
            listData.append(dirData)
            break

        # Si todas son nan no sirve
        L = 0
        for index, v_reg in enumerate(varible_v):
            if str(df[v_reg].values[i]) == "nan":
                L = L+1
        if L == len(varible_v):
            print("allNone", anio)
            continue

        if anioInit < anio:
            if anioInit > 0:
                listData.append(dirData)
                dirData = {}
                initDir = True

        if initDir:
            dirData["anio"] = anio
            dirData["regiones"] = {}
            for index, v_reg in enumerate(varible_v):
                dirData["regiones"][reg_name[index]] = {"meses": {}}

            initDir = False

        for index, v_reg in enumerate(varible_v):
            dirData["regiones"][reg_name[index]]["meses"][mes] = {}

        for index, v_reg in enumerate(varible_v):

            # dirData["regiones"][reg_name[index]]["meses"][mes]["indicar_principal"]=df[v_reg].values[i]
            if norm:
                dirData["regiones"][reg_name[index]]["meses"][mes][variable_meaning[index]] = round(
                    float(df_norm[v_reg].values[i]), 4)
                if (df_norm[v_reg].values[i]) == "nan":
                    del dirData["regiones"][reg_name[index]]["meses"][mes]
                    break

            else:
                # si no hay data
                data_value = repair_num(df[v_reg].values[i])
                """
                try:
                    data_value=float(df[v_reg].values[i])
                except:
                    "Existe algun signo"
                    aux=""
                    for letra in str(df[v_reg].values[i]):
                        if letra.isdigit():
                            aux=aux+letra
                        elif letra ==".":
                            aux=aux+letra
                            
                    data_value=float(aux)
                
                """

                if str(data_value).lower() == "nan" or len(str(data_value).lower()) == 0:
                    data_value = 0
                if data_value - int(data_value) > 0.0001:
                    data_value = round(data_value, 1)
                else:
                    data_value = int(data_value)
                dirData["regiones"][reg_name[index]
                                    ]["meses"][mes][variable_meaning[index]] = data_value
                if (df[v_reg].values[i]) == "nan":
                    del dirData["regiones"][reg_name[index]]["meses"][mes]
                    break

            if len(toolTip_v[index]) > 0:

                toolTip_values = [str(df[x].values[i])
                                  for x in toolTip_v[index]]
                tip = {}
                #print(index, v_reg, i, df[v_reg].values[i])
                data_value = repair_num(df[v_reg].values[i])

                """
                try:
                    data_value=float(df[v_reg].values[i])
                except:
                    "Existe algun signo"
                    aux=""
                    for letra in str(df[v_reg].values[i]):
                        if letra.isdigit():
                            aux=aux+letra
                        elif letra ==".":
                            aux=aux+letra
                            
                    data_value=float(aux)
                
                """

                if str(data_value).lower() == "nan" :
                    data_value = "No Data"
                else:
                    if data_value - int(data_value) > 0.0001:
                        data_value = round(data_value, 1)
                    else:
                        data_value = int(data_value)
                tip[variable_meaning[index]] = data_value

                # add data to tooltipe

                tip["Fecha: "] = str(int(df["anio"].values[i])) + \
                    "/ " + df["mesSTR"].values[i]

                for indexTool, name in enumerate(toolTip_mening[index]):
                    if str(toolTip_values[indexTool]).lower() == "nan":
                        tip[name] = "-"
                    else:
                        tip[name] = toolTip_values[indexTool]

                dirData["regiones"][reg_name[index]
                                    ]["meses"][mes]["tooltip"] = tip
            else:

                tip = {}
                data_value = repair_num(df[v_reg].values[i])
                if str(data_value).lower() == "nan" :
                    data_value = "No Data"
                else:
                    if data_value - int(data_value) > 0.0001:
                        data_value = round(data_value, 1)
                    else:
                        data_value = int(data_value)
                tip[variable_meaning[index]] = data_value

                tip["Fecha: "] = str(int(df["anio"].values[i])) + \
                    "/ " + df["mesSTR"].values[i]
                
                dirData["regiones"][reg_name[index]
                                    ]["meses"][mes]["tooltip"] = tip
                #old
                #dirData["regiones"][reg_name[index]]["meses"][mes]["tooltip"] ={}
        anioInit = anio

    return listData

## D. Creating image to download
sns.set()
def create_image(df, varible_v, variable_meaning, save_dir):
    print("img", varible_v, variable_meaning)
    objetive_var = varible_v

    plt.close()
    plt.clf()
    
    df['date'] = df["Mes"].map(str) + '-' + df["anio"].map(str)
    for value in df['date'] :
        df['date'] = pd.to_datetime(
        df['date'], format='%m-%Y').dt.strftime('%m-%Y')

    date = []
    val = []
    for i, value in enumerate(df[objetive_var]):
        if str(value) == "nan":
            continue
        else:
            value = repair_num(value)
            val.append(value)
            print((df['date'].values[i]))
            date.append(df['date'].values[i])

    df = pd.DataFrame(columns=['date', objetive_var])
    df['date'] = date
    df[objetive_var] = val

    fig, ax = plt.subplots()
    sns.set_style("darkgrid")

    print(len(df['date']), len(df[objetive_var]))

    ax = sns.lineplot(x=range(0, len(df['date'])), y=df[objetive_var])

    """
    max_xticks = 11
    xloc = plt.MaxNLocator(max_xticks)
    print(xloc)
    ax.xaxis.set_major_locator(xloc)
    """

    names = []
    print(len(df['date']))

    ran = int((len(df['date'])+.5)/10)
    if ran == 0:
        ran = 1
    if len(df['date']) / ran < 10:
        ran = 1
    names = [df['date'].values[d] for d in range(0, len(df['date']), ran)]
    ticklabels = names
    ax.set_xticks(range(0, len(df['date'])+1, ran))
    ax.set_xticklabels(ticklabels)
    fig.autofmt_xdate()
    plt.title(variable_meaning)
    plt.ylabel(variable_meaning)
    plt.savefig(save_dir, dpi=100)
    plt.show()
    
def naming_image(x):
    return global_dir + "/img/" + init_data_name + x + '.png'

def naming_file(y):
    return global_dir + "/" + init_data_name + y + '.json'      
    
## E. Reading Data from API 
file = 'Innovacion.xlsx'
global_dir = "./data"
norm = False  # no normalizada
init_data_name="Inov_"
link_data = "https://smartdata-demo.sfo2.digitaloceanspaces.com/data/Innovacion.xlsx"
link_img = "https://smartdata-demo.sfo2.digitaloceanspaces.com/data/img/"
link_glosario="https://smartdata-demo.sfo2.digitaloceanspaces.com/data/"
info_header=[ {"titulo":None,"Fuente":None, "LinkDatosOriginales":None, 
               "comment":None, "linkPlot":None,"glosario":None}]     

## Processing Data 

### 1) Inov_01_Tasa_inn.json

In [2]:
sheetname ="TasaInnovacion"
full_file_name = '01_Tasa_inn'
info_header[0]["titulo"] = "Tasa de Innovación en la Industria de la Construcción"
info_header[0]["Fuente"] = "Encuesta innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] = "Tasa de Innovación considera las empresas del sector que innovaron en los años correspondientes a los datos de la Encuesta de Innovación del Ministerio de Economía, Fomento y Turismo. La encuesta 2016 representa el levantamiento de datos de años 2014 y 2015"
info_header[0]["linkPlot"] =""
info_header[0]["glosario"] = ""

if sheetname == "TasaInnovacion":
    df = pd.read_excel(file, sheet_name=sheetname)
    data = [["INOVARON"]]
    dataCorto = [["INOVARON [%]"]]

    for index, lista in enumerate(data):
        toolTip_v = [[]]
        toolTip_mening = [[]]
        reg_name = ["PAIS"]
        for index2, varname in enumerate(lista):
            if index2 == 0:
                varible_v = [varname]
                variable_meaning = [dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening, norm)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_01_Tasa_inn.json


### 2) Inov_02_gasto_inn.json

In [3]:
sheetname ="GastoInnovacion"
full_file_name = '02_gasto_inn'
info_header[0]["titulo"] = "Gasto en Innovación en relación a las ventas sector construcción"
info_header[0]["Fuente"] = "Encuesta innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] = "Considere los gastos declarados por las empresas en cuanto a innovación sobre sus ventas. Fuente Encuesta de Innovación del Ministerio de Economía, Fomento y Turismo. La encuesta 2016 representa el levantamiento de datos de años 2014 y 2015"
info_header[0]["linkPlot"] =""
info_header[0]["glosario"] = ""

if sheetname == "GastoInnovacion":
    df = pd.read_excel(file, sheet_name=sheetname)
    data = [["Porc inovacion"]]
    dataCorto = [["Gasto [%]"]]
    for index, lista in enumerate(data):
        toolTip_v = [[]]
        toolTip_mening = [[]]
        reg_name = ["PAIS"]
        for index2, varname in enumerate(lista):
            if index2 == 0:
                varible_v = [varname]
                variable_meaning = [dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening, norm)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_02_gasto_inn.json


### 3) Inov_03_Cont_inn.json

In [4]:
sheetname = 'Porc_empresas_que_innovan'
full_file_name = '03_Cont_inn'
info_header[0]["titulo"] = "Continuidad de la Innovación"
info_header[0]["Fuente"] = "Encuenta Innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] = "Se consideran el tipo de innovación como continua u ocasional, considerando las empresas que contestaron la encuesta (siempre asociadas a la construcción). Fuente: Encuesta de Innovación del Ministerio de Economía, Fomento y Turismo."
info_header[0]["linkPlot"] = ""
info_header[0]["glosario"] = ""

if sheetname == 'Porc_empresas_que_innovan':
    df = pd.read_excel(file, sheet_name=sheetname)
    varible_v = ["Grande_Porc","mediana_Porc","pequeña_Porc"]
    variable_meaning = ["Grande [%]","Mediana[%]","Pequeña[%]" ]
    reg_name = ["Grande","Mediana","Pequeña" ]
    toolTip_v = [[],[],[]]
    toolTip_mening =[[],[],[]]

    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_03_Cont_inn.json


### 4) Inov_04_Nota_Costos.json

In [5]:
sheetname ="Nota Factores innovacion"
full_file_name = '04_Nota_Costos'
info_header[0]["titulo"] = "Nota Costos"
info_header[0]["Fuente"] = "Encuesta innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] = info_header[0]["titulo"] 
info_header[0]["linkPlot"] =""
info_header[0]["glosario"] = ""

if sheetname == "Nota Factores innovacion":
    df = pd.read_excel(file, sheet_name=sheetname)
    data = [["Nota Costos"]]
    dataCorto = [["Nota Costos"]]
    for index, lista in enumerate(data):
        toolTip_v = [[]]
        toolTip_mening = [[]]
        reg_name = ["PAIS"]
        for index2, varname in enumerate(lista):
            if index2 == 0:
                varible_v = [varname]
                variable_meaning = [dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening, norm)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_04_Nota_Costos.json


### 5) Inov_05_Nota_conocimiento.json

In [6]:
sheetname ="Nota Factores innovacion"
full_file_name = '05_Nota_conocimiento'
info_header[0]["titulo"] = "Nota Conocimiento"
info_header[0]["Fuente"] = "Encuesta innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] =info_header[0]["titulo"] 
info_header[0]["linkPlot"] = ""
info_header[0]["glosario"] = ""

if sheetname == "Nota Factores innovacion":
    df = pd.read_excel(file, sheet_name=sheetname)
    data = [["Nota conocimiento"]]
    dataCorto = [["Nota Conocimiento"]]
    for index, lista in enumerate(data):
        toolTip_v = [[]]
        toolTip_mening = [[]]
        reg_name = ["PAIS"]
        for index2, varname in enumerate(lista):
            if index2 == 0:
                # Nombre de la variable
                varible_v = [varname]
                variable_meaning = [dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening, norm)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_05_Nota_conocimiento.json


### 6) Inov_06_Nota_Mercado.json

In [7]:
sheetname ="Nota Factores innovacion"
full_file_name = '06_Nota_Mercado'
info_header[0]["titulo"] = "Nota Mercado"
info_header[0]["Fuente"] = "Encuesta innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] =info_header[0]["titulo"] 
info_header[0]["linkPlot"] = ""
info_header[0]["glosario"] = ""

if sheetname == "Nota Factores innovacion":
    df = pd.read_excel(file, sheet_name=sheetname)
    data = [["Nota Mercado"]]
    dataCorto = [["Nota Mercado"]]
    for index, lista in enumerate(data):
        toolTip_v = [[]]
        toolTip_mening = [[]]
        reg_name = ["PAIS"]
        for index2, varname in enumerate(lista):
            if index2 == 0:
                # Nombre de la variable
                varible_v = [varname]
                variable_meaning = [dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening, norm)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_06_Nota_Mercado.json


### 7) Inov_07_Nota_otros_factores.json

In [8]:
sheetname ="Nota Factores innovacion"
full_file_name = '07_Nota_otros_factores'
info_header[0]["titulo"] = "Nota Mercado: Otros Factores"
info_header[0]["Fuente"] = "Encuesta innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] =info_header[0]["titulo"] 
info_header[0]["linkPlot"] = ""
info_header[0]["glosario"] = ""

if sheetname == "Nota Factores innovacion":
    df = pd.read_excel(file, sheet_name=sheetname)
    data = [["Nota Otros"]]
    dataCorto = [["Nota Otros"]]
    for index, lista in enumerate(data):
        toolTip_v = [[]]
        toolTip_mening = [[]]
        reg_name = ["PAIS"]
        for index2, varname in enumerate(lista):
            if index2 == 0:
                varible_v = [varname]
                variable_meaning = [dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening, norm)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_07_Nota_otros_factores.json


### 8) Inov_08_Inn_futura2.json

In [9]:
sheetname ="InovacionFutura"
full_file_name = '08_Inn_futura2'
info_header[0]["titulo"] = "Predisposición a innovar en 2 años"
info_header[0]["Fuente"] = "Encuesta innovación"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] = "Predisposición de las empresas a innovar en 2 años más. Fuente: Encuesta de Innovación"
info_header[0]["linkPlot"] =""
info_header[0]["glosario"] = ""

if sheetname == "InovacionFutura":
    df = pd.read_excel(file, sheet_name=sheetname)
    data = [["Inovacion futura"]]
    dataCorto = [["Inovacion futura"]]
    for index, lista in enumerate(data):
        toolTip_v = [[]]
        toolTip_mening = [[]]
        reg_name = ["PAIS"]
        for index2, varname in enumerate(lista):
            if index2 == 0:
                varible_v = [varname]
                variable_meaning = [dataCorto[index][index2]]
            else:
                toolTip_v[0].append(varname)
                toolTip_mening[0].append(dataCorto[index][index2])
                
    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening, norm)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_08_Inn_futura2.json


### 9) Inov_09_Bim_inst.json

In [2]:
sheetname = 'Intitucion'
file='BIM.xlsx'
full_file_name = '09_Bim_inst'
info_header[0]["titulo"] = "Instituciones con capacitación formal de BIM por tipo de institución y nivel de carrera"
info_header[0]["Fuente"] = "PlanBIM"
info_header[0]["LinkDatosOriginales"] = link_data
info_header[0]["comment"] = "Cantidad de carreras que con formación profesional BIM por tipo de institución y nivel de carrera. Fuente: Plan BIM"
info_header[0]["linkPlot"] = ""
info_header[0]["glosario"] = ""

if sheetname == 'Intitucion':
    df = pd.read_excel(file, sheet_name=sheetname)
    varible_v = [ 'total','Universidades Privadas ',
       'Universidades Estatales', 'Institutos Profesionales',
       'Centros de Formación Técnica', 'Empresas']
    variable_meaning = ["total" ,"U. Privadas","U. Estatales","Inst. Prof.",
        "Form. Técnica","Empresas"]
    reg_name = ["total" ,"U. Privadas","U. Estatales","Inst. Prof.",
        "Form. Técnica","Empresas"]
    toolTip_v = [["var_total"],
    ['var_Universidades Privadas '],
    ["var_Universidades Estatales"],
    ["var_Institutos Profesionales"],
    ["var_Centros de Formación Técnica"],
    ["var_Empresas"]]
    toolTip_mening =[["Var Anual"],["Var Anual"],["Var Anual"],
                    ["Var Anual"],["Var Anual"],["Var Anual"]]


    jsonDir = DataToJson(df, varible_v, variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_09_Bim_inst.json


### 10) Inov_10_Bim_inst_region.json

In [3]:
sheetname = 'region'
file='BIM.xlsx'
full_file_name = '10_Bim_inst_region'
info_header[0]["titulo"]= "Instituciones con capacitación formal de BIM por tipo de institución y región"
info_header[0]["Fuente"]="ENCUESTA BIM"
info_header[0]["LinkDatosOriginales"]=link_data
info_header[0]["comment"]= "Cantidad de carreras que con formación profesional BIM por tipo de institución y región. Fuente: Plan BIM"
info_header[0]["linkPlot"]=""
info_header[0]["glosario"]=link_glosario+"general.pdf"

if sheetname == 'region':
    df = pd.read_excel(file, sheet_name=sheetname)
    varible_v=["Total general" ]*18
    variable_meaning=["Total general" ]*18
    reg_name=[
    "REGIÓN BIO BIO",
    "REGIÓN DE AISÉN",
    "REGIÓN DE ANTOFAGASTA",
    "REGIÓN DE ARICA Y PARINACOTA",
    "REGIÓN DE COQUIMBO",
    "REGIÓN DE LA ARAUCANÍA",
    "REGIÓN DE LOS LAGOS",
    "REGIÓN DE LOS RÍOS",
    "REGIÓN DE MAGALLANES Y ANTÁRTICA CHILENA",
    "REGIÓN DE ÑUBLE",
    "REGIÓN DE TARAPACÁ",
    "REGIÓN DE VALPARAÍSO",
    "REGIÓN DEL BIO BIO",
    "REGIÓN DEL MAULE",
    "REGIÓN LIBERTADOR GENERAL BERNARDO O'HIGGINS",
    "REGIÓN METROPOLITANA",
    "REGIÓON DEL MAULE",
    "REGIÓN DE ATACAMA",
    ]

    toolTip_v=[["Postgrado",	"Postítulo",	"Pregrado"]]*18
    toolTip_mening=[["Postgrado",	"Postítulo",	"Pregrado"] ]*18

    jsonDir = DataToJson(df,varible_v,variable_meaning,reg_name, toolTip_v, toolTip_mening)
    jsonDir = [info_header, jsonDir]
    jsonUTF8 = json.dumps(jsonDir, ensure_ascii=False).encode('utf8')
    jsonUTF8 = json.loads(jsonUTF8)    
#     create_image(df, varible_v[0], variable_meaning[0],naming_image(full_file_name))
    name_file = naming_file(full_file_name)
    with open(name_file , 'w') as outfile:  
        json.dump(jsonUTF8, outfile,ensure_ascii=False)
    print("CREATED: ", name_file)     

set: None
CREATED:  ./data/Inov_10_Bim_inst_region.json
